# `grib_to_netcdf` Processing of Continental HRDPS with `pywgrib2_xr`

Exploration of doing the processing of the SalishSeaNowcast `grib_to_netcdf` worker
to generate NEMO atmospheric forcing files for SalishSeaCast from the
ECCC MSC 2.5 km rotated lat-lon continental grid HRDPS GRIB2 files using the `pywgrib2_xr` package.

`pywgrib2_xr`:
* code: https://github.com/yt87/pywgrib2_xr
* docs: https://yt87.github.io/pywgrib2_xr/index.html

`conda` environment description for this notebook: `analysis-doug/notebooks/continental-HRDPS/environment.yaml`

**Notes:**
* the `pywgrib2_xr` conda package is published on the `yt87` channel, not on `conda-forge`
* it has important dependency limits that are *not* specified in the published package:
  * installation fails for any Python version other than 3.9
  * there is a runtime `ImportError`:

         ImportError: cannot import name 'dask_array_type' from 'xarray.core.pycompat'
  
    with 26-Feb-2023 versions of `dask` and `xarray`
  * based on the 23-Mar-2021 publication date of the `ty87/pywgrib2_xr`, I pinned `dask` and `xarray` at:
    * `dask=2021.03.0`
    * `xarray=0.17.0`

This notebook assumes that the MOAD `/results/` file system is mounted.

## Play with the Low Level `wgrib2` Interface

`pywgrib2_xr.wgrib()` is a wrapper for the
[wgrib2](https://www.cpc.ncep.noaa.gov/products/wesley/wgrib2/long_cmd_list.html)
command-line tool

In [1]:
import os
from pathlib import Path

import pywgrib2_xr

In [2]:
west_files = Path("/results/forcing/atmospheric/GEM2.5/GRIB/20230215/00/001/")
conti_files = Path("/results/forcing/atmospheric/continental2.5/GRIB/20230215/00/001/")

west_u_file = west_files / "CMC_hrdps_west_UGRD_TGL_10_ps2.5km_2023021500_P001-00.grib2"

conti_u_file = conti_files / "20230215T00Z_MSC_HRDPS_UGRD_AGL-10m_RLatLon0.0225_PT001H.grib2"
conti_v_file = conti_files / "20230215T00Z_MSC_HRDPS_VGRD_AGL-10m_RLatLon0.0225_PT001H.grib2"

In [3]:
pywgrib2_xr.wgrib(west_u_file)
pywgrib2_xr.free_files(os.fspath(west_u_file))

1:0:d=2023021500:UGRD:10 m above ground:60 min fcst:


**Note:** `pywgrib2_xr.wgrib()` calls aren't repeatable unless the file is freed with `free_files()`.

I feel a context manager coming on...

### Get grid descriptions:

In [4]:
args = "-d 1 -grid"
pywgrib2_xr.wgrib(*args.split(), west_u_file)
pywgrib2_xr.free_files(os.fspath(west_u_file))

1:0:grid_template=20:winds(grid):
	polar stereographic grid: (685 x 485) input WE:SN output WE:SN res 8
	North pole lat1 44.689624 lon1 230.093688 latD 60.000000 lonV 247.000000 dx 2500.000000 m dy 2500.000000 m


In [5]:
args = "-d 1 -grid"
pywgrib2_xr.wgrib(*args.split(), conti_u_file)
pywgrib2_xr.free_files(os.fspath(conti_u_file))

1:0:grid_template=1:winds(grid):
	rotated lat-lon grid:(2540 x 1290) units 1e-06 input WE:SN output WE:SN res 56
	lat -12.302501 to 16.700001 by 0.022500
	lon 345.178780 to 42.306283 by 0.022500 #points=3276600
	south pole lat=-36.088520 lon=245.305142 angle of rot=0.000000


For the rotated lat-lon grid, the grid description string that is required to do remapping
(e.g. rotation of winds) has the format:

        rot-ll:sp_lon:sp_lat:sp_rot lon0:nlon:dlon lat0:nlat:dlat

where:

        sp_lon = longitude of the South pole (for rotation)
        sp_lat = latitude of the South pole (for rotation)
        sp_rot = angle of rotation (degrees)
 
        lat0, lon0 = degrees of lat/lon for 1st grid point 
        nlat = number of longitudes
        nlon = number of latitudes
        dlon = grid cell size in degrees of longitude
        dlat = grid cell size in degrees of latitude

See: https://www.cpc.ncep.noaa.gov/products/wesley/wgrib2/new_grid.html

There does not appear to be a code way to substitute in values from the grid description above.
Doing it by inspection gives:

In [6]:
conti_grid_desc = (
    "rot-ll:245.305142:-36.088520:0.000000 345.178780:2540:0.022500 -12.302501:1290:0.022500"
)

### `grib_to_netcdf._rotate_grib_wind()` Processing

2 steps:
1) create a GRIB2 file containing the UGRD and VGRD variables
2) remap that file from the model grid to an u-goes-east, v-goes-north grid

In [7]:
in_files = (conti_u_file, conti_v_file)
out_file = Path("/tmp", "uv_py.grib")
out_file.unlink(missing_ok=True)
args = f"-append -grib {out_file}"

for f in in_files:
    pywgrib2_xr.wgrib(f, *args.split())
    pywgrib2_xr.free_files(os.fspath(f))

!ls -lh /tmp/*.grib

1:0:d=2023021500:UGRD:10 m above ground:60 min fcst:
1:0:d=2023021500:VGRD:10 m above ground:60 min fcst:
-rw-rw-r-- 1 doug doug 3.6M Feb 27 17:21 /tmp/uv_py.grib
-rw-rw-r-- 1 doug doug    0 Feb 27 17:21 /tmp/uvrot_py.grib


In [8]:
in_file = out_file
out_file = Path( "/tmp", "uvrot_py.grib")
out_file.unlink(missing_ok=True)
args = f"-new_grid_winds earth -new_grid {conti_grid_desc}"

pywgrib2_xr.wgrib(in_file, *args.split(), out_file)
pywgrib2_xr.free_files(os.fspath(in_file), os.fspath(out_file))

!ls -lh /tmp/*.grib

1:0:d=2023021500:UGRD:10 m above ground:60 min fcst:
2:1827917:d=2023021500:VGRD:10 m above ground:60 min fcst:
-rw-rw-r-- 1 doug doug 3.6M Feb 27 17:21 /tmp/uv_py.grib
-rw-rw-r-- 1 doug doug 9.4M Feb 27 17:21 /tmp/uvrot_py.grib
